In [1]:
import json
import os
import pandas as pd
from src.RAG_dataset_generator import RAGDatasetGenerator

In [2]:
CHUNKS_FOLDER = "../../data\D26\ladrome-2023-12-22-page-chunked-v1"

In [9]:
handmade_dataset = pd.read_excel("../../data/D26/D26_dataset_handmade.xlsx")
handmade_dataset["relevant_chunk_ids"] = handmade_dataset["relevant_chunk_ids"].str.split(" ")
handmade_dataset.dropna(subset="relevant_chunk_ids", inplace=True)

filenames = handmade_dataset["relevant_chunk_ids"].tolist()
filenames = [fn[0] for fn in filenames]

dataset_generator = RAGDatasetGenerator("mistral", model="mistral_small")
dataset_generator.generate_RAG_dataset(CHUNKS_FOLDER, filenames, max_retries=5)

TypeError: RAGDatasetGenerator.generate_RAG_dataset() got an unexpected keyword argument 'max_retries'